In [2]:
import torchvision
import torchvision.datasets as datasets
import torch
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import tqdm
import mod_seqs, FileUtils, constants
from SeqsData import SeqsData
import numpy as np




In [3]:
# check if GPU is available or else default to CPU usage.
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [6]:
# One hot encoding 

print("Reading in test, train, and validation files")
test = FileUtils.readFile(constants.TEST) # which is 25% of data
train = FileUtils.readFile(constants.TRAIN) # which is 70% of data
validation = FileUtils.readFile(constants.VALID) # which is 5% of data

print("Splitting the test, train and validation data into lists")
test = [line.split() for line in test]
train = [line.split() for line in train]
validation = [line.split() for line in validation]

print(len(test), len(train), len(validation))



print("\nDone")


Reading in test, train, and validation files
Splitting the test, train and validation data into lists
656590 2626360 172788
One hot encoding 100% of the test data

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 

One hot encoding 100% of the train data

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 

One hot encoding 100% of the validation data

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


Done


In [7]:
# create test dataset to be fed into the dataloader
np.transpose(test_one_hot)
print(test_one_hot.shape)
sd_test = SeqsData(test_one_hot)

# create train dataset to be fed into dataloader
np.transpose(train_one_hot)
sd_train = SeqsData(train_one_hot)

# create train dataset to be fed into dataloader
np.transpose(valid_one_hot)
sd_valid = SeqsData(valid_one_hot)

# DataLoader Objects are what feed the network the data by means of tensor stacks defined in the SeqsData class.
test_dataloader = DataLoader(sd_test, batch_size=256, shuffle=True)
train_dataloader = DataLoader(sd_train, batch_size=256, shuffle=True)
valid_dataloader = DataLoader(sd_valid, batch_size=256, shuffle=True)



(656590, 2)


In [17]:
class   GNet(nn.Module):
    
    def __init__(self):
        
        super().__init__()

        # Layer1
        self.conv1 = nn.Conv1d(in_channels=4, out_channels=96, kernel_size=11, stride=1, padding=1)
    

        # Layer2:
        self.conv2 = nn.Conv1d(in_channels=96, out_channels=96, kernel_size=7, padding=1, stride=2) 
        self.local_response2=nn.LocalResponseNorm(size=5,alpha=0.0001,beta=0.75,k=2)

        # Layer3:
        self.conv3 = nn.Conv1d(in_channels=96, out_channels=96, kernel_size=3, stride=1, padding=1)
        self.pool_layer3 = nn.MaxPool1d(kernel_size=7, stride=2)
        self.local_response = nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2)

        # Layer4:
        self.conv4 = nn.Conv1d(in_channels=96, out_channels=96, kernel_size=7, stride=4, padding=2)

        # Layer5:
        self.conv5=nn.Conv1d(in_channels=96, out_channels=96, kernel_size=1, stride=1)

        # Layer6:
        self.conv6=nn.Conv1d(in_channels=96, out_channels=96, kernel_size=1, stride=2)
        self.pool_layer6=nn.MaxPool1d(kernel_size=1, stride=1) 
       
        
        # Layer7:
        self.conv7=nn.Conv1d(in_channels=96, out_channels=96, kernel_size=3, stride=1, padding=1) 

        # Layer8:
        self.conv8=nn.Conv1d(in_channels=96, out_channels=96, kernel_size=1, stride=1)

        # Layer9:
        self.conv9 = nn.Conv1d(in_channels=96, out_channels=96, kernel_size=2, stride=2)
        
        
        # Layer10:
        self.fc1 = nn.Linear(in_features=(960), out_features=1024)
        
        # Layer11:
        self.fc2 = nn.Linear(in_features=1024, out_features=1024)
             
    def forward(self, x):
        
        # applying ReLU to layer 1
        x = F.relu(self.conv1(x))

        # applying ReLU to layer 2
        x = F.relu(self.conv2(x))

    
        x = self.pool_layer3(F.relu(self.local_response(self.conv3(x))))
        x = F.dropout(x, .5)

        # applying ReLU to layer 4
        x = F.relu(self.conv4(x))

        # applying ReLU to layer 5
        x = F.relu(self.conv5(x))

        
        # applying ReLU to layer 6
        x = F.relu(self.conv6(x))
        x = F.dropout(x, .5)

        # applying ReLU to layer 7
        x = F.relu(self.conv7(x))

        # applying ReLU to layer 8
        x = F.relu(self.conv8(x))

        # applying ReLU to layer 9
        x = F.relu(self.conv9(x))
        x = F.dropout(x, .5)

        x = torch.flatten(x, 1) # flattening all Dimensions
        
        # applying ReLU to layer 10
        x = F.relu(self.fc1(x))
        x = F.dropout(x, .5)

        # applying ReLU to layer 11
        x = torch.ReLU(self.fc2(x))
        # x = F.dropout(x, .5)

    
          
        return x
        

In [8]:
class AlexNet(torch.nn.Module):
    def __init__(self):
        # result size = ((size - kernelSize + 2 * padding) / stride) + 1
        super().__init__()

        self.conv1 = torch.nn.Conv1d(in_channels=4, out_channels=64, kernel_size=11) 

        self.pool2 = torch.nn.MaxPool1d(kernel_size=3, stride=2)

        self.conv3 = torch.nn.Conv1d(in_channels=64, out_channels=192, kernel_size=5, padding=2) 

        self.pool4 = torch.nn.MaxPool1d(kernel_size=3, stride=2)
        self.conv5 = torch.nn.Conv1d(in_channels=192, out_channels=384, kernel_size=3, padding=1)

        self.conv6 = torch.nn.Conv1d(in_channels=384, out_channels=256, kernel_size=3, padding=1)

        self.conv7 = torch.nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, padding=1) 

        self.pool8 = torch.nn.MaxPool1d(kernel_size=3, stride=2)
        self.pool9 = torch.nn.AdaptiveAvgPool1d(6) 
        self.fc1 = torch.nn.Linear(in_features=(6 * 256), out_features=512)

        self.fc2 = torch.nn.Linear(in_features=512, out_features=1024)


    def forward(self, x):
        x = F.relu(self.conv1(x))

        x = self.pool2(x)

        x = F.relu(self.conv3(x))

        x = self.pool4(x)

        x = F.relu(self.conv5(x))

        x = F.relu(self.conv6(x))

        x = F.relu(self.conv7(x))

        x = self.pool8(x)

        x = self.pool9(x)

        x = torch.flatten(x, 1)

        x = F.dropout(x, 0.5)

        x = F.relu(self.fc1(x))

        x = F.dropout(x, 0.5)

        x = F.relu(self.fc2(x))
        
        
        
        return x

In [9]:
# Instantiating the model 
model = AlexNet()
model.load_state_dict(torch.load("./alexnet.pth"))
model = model.to(device)
optimizer = optim.Adam(params=model.parameters(),lr=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [10]:
def processTraining(model,device, train_dataloader, optimizer, epochs):
    model.train()
    for batch_ids, (seq, classes) in tqdm(enumerate(train_dataloader)):
        classes=classes.type(torch.LongTensor)
        seq, classes=seq.to(device), classes.to(device)
        torch.autograd.set_detect_anomaly(True)     
        optimizer.zero_grad()
        output=model(seq)
        loss = loss_fn(output,classes)            
        loss.backward()
        optimizer.step()

In [11]:
def test(model, device, test_dataloader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for seq,classes in tqdm(test_dataloader):
            seq,classes=seq.to(device), classes.to(device)
            y_hat=model(seq)
            test_loss+=F.nll_loss(y_hat,classes,reduction='sum').item()
            _,y_pred=torch.max(y_hat,1)
            correct+=(y_pred==classes).sum().item()
    
        test_loss/=len(test_dataloader)
        print("\n Test set: Average loss: {:.0f},Accuracy:{}/{} ({:.0f}%)\n".format(
            test_loss,correct,len(test_dataloader.dataset),100.*correct/len(test_dataloader.dataset)))
        print('*'*50)


In [12]:
def confusion_matrix(preds, classes, tp=0, tn=0, fp=0, fn=0):
    for pred, cl in zip(preds, classes):
        if pred == 1 and cl == 1:
            tp += 1
        if pred == 0 and cl == 1:
            fn += 1
        if pred ==  1 and cl == 0:
            fp += 1
        if pred == 0 and cl == 0:
            tn += 1
    return tp, tn, fp, fn

In [13]:
ef validation(model, device, valid_dataloader):
    model.eval()
    test_loss=0
    correct=0
    tp, tn, fp, fn = 0, 0, 0, 0
    
    with torch.no_grad():
        for seq,classes in valid_dataloader:
            seq,classes=seq.to(device), classes.to(device)
            y_hat=model(seq)
            test_loss+=F.nll_loss(y_hat,classes, reduction='sum').item()
            _,y_pred=torch.max(y_hat, 1)
            correct+=(y_pred==classes).sum().item()
            tp, tn, fp, fn = confusion_matrix(y_pred, classes, tp, tn, fp, fn)
        test_loss/=len(valid_dataloader)
        print("\n Valid_set: Average loss: {:.0f},Accuracy:{}/{} ({:.0f}%)\n".format(
                test_loss,correct,len(valid_dataloader.dataset),100.*correct/len(valid_dataloader.dataset)))
        print('='*30)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)
    print(f"Actual no total: {tn + fp}, actual yes total: {fn + tp}")
    print(f"n = {tp + tn + fp + fn}")
    print(f"Precision: {precision}, Recall: {recall}, F1 Score: {f1}")
    print(f"True positives: {tp}, True negatives: {tn}, False positives: {fp}, False negatives: {fn}")
    print(f"Predicted no total: {tn + fn}, predicted yes total: {tp + fp}")
        
        

In [14]:
if __name__=='__main__':
    seed=42
    EPOCHS=2 
    
    for epoch in range(1,EPOCHS+1):
        train(model,device,train_dataloader,optimizer,epoch)
        test(model,device,test_dataloader)
        validation(model, device, valid_dataloader)
        torch.save(model.state_dict(), constants.ALEXNETSAVE + f"{epoch + 2}" + ".pth") # save model for each epoch

Train Epoch: 1 [1600248/2626360 (100%)]	Loss: 0.140855

 Test set: Average loss: -1310,Accuracy:626382/656590 (65%)


 Validation set: Average loss: -1321,Accuracy:168362/172788 (67%)

n = 172788
True positives: 81867, True negatives: 75123, False positives: 2460, False negatives: 5783
Actual no total: 87594, actual yes total: 86394
Predicted no total: 89650, predicted yes total: 83138
Precision: 0.9713488416849094, Recall: 0.934740838484154, F1 Score: 0.9526932968407145
Train Epoch: 2 [1600248/2626360 (100%)]	Loss: 0.086065

 Test set: Average loss: -1358,Accuracy:635814/656590 (77%)


 Validation set: Average loss: -1356,Accuracy:166582/172788 (76%)

n = 172788
True positives: 83861, True negatives: 82721, False positives: 3673, False negatives: 2533
Actual no total: 86394, actual yes total: 86394
Predicted no total: 85254, predicted yes total: 87534
Precision: 0.9580391619256517, Recall: 0.970680834317198, F1 Score: 0.9643185686031002
Train Epoch: 3 [1600248/2626360 (100%)]	Loss: 0.

In [ ]:
# torch.save(model.state_dict(), "zachnetfinal.pth") # save final

In [ ]:
model = torch.load("zachnet.pth")
print(model)
# model.eval()

OrderedDict([('conv1.weight', tensor([[[-0.1301, -0.0414, -0.1000,  ...,  0.1285, -0.0584,  0.0390],
         [-0.0343, -0.0470, -0.0336,  ...,  0.1267, -0.1360, -0.0865],
         [-0.0023, -0.1100,  0.0172,  ...,  0.0850, -0.1270, -0.1141],
         [-0.0009,  0.0879, -0.0347,  ...,  0.1460,  0.0240,  0.1367]],

        [[-0.0524, -0.0461,  0.0385,  ..., -0.1278,  0.0421,  0.0551],
         [-0.0746,  0.0903,  0.0191,  ..., -0.0905,  0.0104, -0.0546],
         [ 0.1283,  0.1435,  0.0272,  ..., -0.0091, -0.1077,  0.0433],
         [-0.0999,  0.0991,  0.0870,  ..., -0.0262,  0.1332,  0.0667]],

        [[ 0.1063,  0.0370, -0.0834,  ..., -0.1467,  0.1038,  0.1239],
         [-0.0427, -0.1389,  0.1516,  ...,  0.1308,  0.0983,  0.1401],
         [ 0.0877,  0.1097, -0.0367,  ..., -0.1066, -0.1345,  0.0721],
         [ 0.1320,  0.0412, -0.0722,  ..., -0.0639,  0.1087, -0.0815]],

        ...,

        [[-0.1315, -0.0685, -0.1925,  ...,  0.0684,  0.0901, -0.0424],
         [ 0.0899,  0.0967,